In [0]:
import torch
import torchtext
from torchtext import data

In [0]:
TEXT = data.Field(include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_data = data.TabularDataset('/content/gdrive/My Drive/Colab Notebooks/pytorch/nsmc/ratings_train.json', 
                                'json', {'document': ('text', TEXT), 'label':('label', LABEL)})
test_data = data.TabularDataset('/content/gdrive/My Drive/Colab Notebooks/pytorch/nsmc/ratings_test.json', 
                                'json', {'document': ('text', TEXT), 'label':('label', LABEL)})

In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(0))

In [6]:
print(len(train_data))
print(len(valid_data))

105000
45000


In [0]:
MAX_VOCAB_SIZE = 75000

nsmc_vector = torchtext.vocab.Vectors("/content/gdrive/My Drive/Colab Notebooks/pytorch/nsmc/nsmc_vec_300d.txt", unk_init = torch.Tensor.normal_) 

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE,
                 vectors = nsmc_vector)

LABEL.build_vocab(train_data)

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size = BATCH_SIZE, device = device, sort_within_batch = True, sort_key = lambda x: len(x.text))

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,
                 num_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                           bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(output_dim)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        return self.bn(self.fc(hidden))
    

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
NUM_LAYERS = 2
BIDIRECTIONAL = True

In [0]:
model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)
model = model.to(device)

In [0]:
def count_parameters(model):
     return sum(p.numel() for p in model.parameters() if p.requires_grad) 

In [0]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

In [0]:
#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True)
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [0]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

In [0]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
best_valid_loss = 1.0

for epoch in range(5):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    scheduler.step() #lr scheduler

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    print(f'{epoch+1} : train {train_loss}, {train_acc*100:.2f}% | valid {valid_loss}, {valid_acc*100:.2f}%')

1 : train 0.40908247245149887, 82.55% | valid 0.34394662909802387, 85.55%
2 : train 0.31575117237062295, 86.71% | valid 0.3198998527897691, 86.37%
3 : train 0.2648866368613179, 89.26% | valid 0.3153182525949722, 86.73%
4 : train 0.23990020426733694, 90.44% | valid 0.32117824544283474, 86.66%
5 : train 0.2111841556885706, 91.86% | valid 0.3397106087068096, 86.81%


In [22]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'{test_loss}, {test_acc*100:.2f}%')

0.3182620618997328, 86.57%


In [0]:
from google.colab import files

files.download('/content/model.pt')